In [1]:
import os
# comment out below line to enable tensorflow logging outputs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import time
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
from absl import app, flags, logging
from absl.flags import FLAGS
import core.utils as utils
from core.yolov4 import filter_boxes
from tensorflow.python.saved_model import tag_constants
from core.config import cfg
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
# deep sort imports
from deep_sort import preprocessing, nn_matching
from deep_sort.detection import Detection
from deep_sort.tracker import Tracker
from tools import generate_detections as gdet


import torch
import math

C:\Users\keven\anaconda3\envs\yolov4-gpu2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#load yolov5 for object detection
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # or yolov5m, yolov5l, yolov5x, custom

Using cache found in C:\Users\keven/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-4-22 torch 1.11.0 CUDA:0 (NVIDIA GeForce GTX 1050, 2048MiB)

Fusing layers... 
YOLOv5s_v6 summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [3]:
# yolov5 model settings:
model.classes = [0] # only detect humans -> class = 0
model.max_det = 1

In [4]:
#Pose Classification Model
interpreter = tf.lite.Interpreter(model_path='data/lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

In [5]:
def get_bbox(results, frame): #pd is a 
    pd = results.pandas().xyxy[0].sort_values('confidence')
    
    nb_detected_objs = len(pd.index)
    
    bbox = np.zeros((4,nb_detected_objs),dtype=int)#4 rows for x0,y0,x1,y1 --> each column is the bb of 1 img
    confidence = np.zeros(nb_detected_objs)
    class_names = np.zeros(nb_detected_objs,dtype=object)
    images = []
    
    counter = 0
    for obj in pd.iloc:
        x0, y0, x1, y1 = obj.to_numpy()[0:4].astype(int)
        images.append(frame[y0:y1, x0:x1])
        confidence[counter] = obj.to_numpy()[4]
        class_names[counter] = obj.to_numpy()[6]
        #print(obj.to_numpy()[6])
        bbox[:,counter] = np.array([x0,y0,x1,y1],dtype=int)
        counter += 1
    
    #print(class_names)
    return bbox, confidence, class_names, images

def format_boxes(bboxes):#for deep sort
    for box in bboxes:
        ymin = int(box[1])
        xmin = int(box[0])
        ymax = int(box[3])
        xmax = int(box[2])
        width = xmax - xmin
        height = ymax - ymin
        box[0], box[1], box[2], box[3] = xmin, ymin, width, height
    return bboxes

# Functions used for Pose Classification

## MoveNet output:

A float32 tensor of shape [1, 1, 17, 3].

● The first two channels of the last dimension represents the yx coordinates (normalized to
image frame, i.e. range in [0.0, 1.0]) of the 17 keypoints (in the order of: [nose, left eye,
right eye, left ear, right ear, left shoulder, right shoulder, left elbow, right elbow, left wrist,
right wrist, left hip, right hip, left knee, right knee, left ankle, right ankle]).

● The third channel of the last dimension represents the prediction confidence scores of
each keypoint, also in the range [0.0, 1.0]

In [6]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

def calculateAngle(landmark1, landmark2, landmark3):
    '''
    This function calculates angle between three different landmarks.
    Args:
        landmark1: The first landmark containing the x,y and z coordinates.
        landmark2: The second landmark containing the x,y and z coordinates.
        landmark3: The third landmark containing the x,y and z coordinates.
    Returns:
        angle: The calculated angle between the three landmarks.
    '''

    # Get the required landmarks coordinates.
    x1, y1 = landmark1
    x2, y2 = landmark2
    x3, y3 = landmark3

    # Calculate the angle between the three points
    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
    angle = np.abs(angle)
    # Check if the angle is less than zero.
    if angle > 180.0:

        angle = 360-angle
        
    # Return the calculated angle.
    return angle

def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) 
            
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)
            
def classifyPose(kp_array, output_image, display=False):
    '''
    This function classifies yoga poses depending upon the angles of various body joints.
    Args:
        kp_array: A list of detected landmarks of the person whose pose needs to be classified.
        output_image: A image of the person with the detected pose landmarks drawn.
        display: A boolean value that is if set to true the function displays the resultant image with the pose label 
        written on it and returns nothing.
    Returns:
        output_image: The image with the detected pose landmarks drawn and pose label written.
        label: The classified pose label of the person in the output_image.

    '''
    
    # Initialize the label of the pose. It is not known at this stage.
    label = 'Unknown Pose'

    # Specify the color (Red) with which the label will be written on the image.
    color = (0, 0, 255)
    
    # Calculate the required angles.
    #----------------------------------------------------------------------------------------------------------------
    
    # Get the angle between the left shoulder, elbow and wrist points. 
    left_elbow_angle = calculateAngle(kp_array[5],
                                      kp_array[7],
                                      kp_array[9])
    
    # Get the angle between the right shoulder, elbow and wrist points.
    right_elbow_angle = calculateAngle(kp_array[6],
                                       kp_array[8],
                                       kp_array[10])
    
    # Get the angle between the left elbow, shoulder and hip points.
    left_shoulder_angle = calculateAngle(kp_array[7],
                                         kp_array[5],
                                         kp_array[11])
    # Get the angle between the right hip, shoulder and elbow points.
    right_shoulder_angle = calculateAngle(kp_array[12],
                                          kp_array[6],
                                          kp_array[8])
    
    #print('left_elbow_angle: ', left_elbow_angle ,'\n right_elbow_angle: ', right_elbow_angle)
    #print('left_shoulder_angle: ', left_shoulder_angle ,'\n right_shoulder_angle: ', right_shoulder_angle)
    
    #----------------------------------------------------------------------------------------------------------------
    
    # Check if it is the warrior II pose or the T pose.
    # As for both of them, both arms should be straight and shoulders should be at the specific angle.
    #----------------------------------------------------------------------------------------------------------------
    
    # Check if the both arms are straight.
    if left_elbow_angle > 125 and left_elbow_angle < 220 and right_elbow_angle > 125 and right_elbow_angle < 220:
        #label = 'T Pose'
        # Check if shoulders are at the required angle.
        if left_shoulder_angle > 70 and left_shoulder_angle < 110 and right_shoulder_angle > 70 and right_shoulder_angle < 110:
            label = 'T Pose'
                        
    if right_elbow_angle > 50 and right_elbow_angle < 130 and right_shoulder_angle > 70 and right_shoulder_angle < 110:
        label = "power to the people"
    # Check if the pose is classified successfully
    if label != 'Unknown Pose':
        
        # Update the color (to green) with which the label will be written on the image.
        color = (0, 255, 0)  
    
    # Write the label on the output image. 
    cv2.putText(output_image, label, (10, 30),cv2.FONT_HERSHEY_PLAIN, 1, color, 2)
    
    # Check if the resultant image is specified to be displayed.
    if display:
    
        # Display the resultant image.
        plt.figure(figsize=[10,10])
        plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');
        
    else:
        
        # Return the output image and the classified label.
        return output_image, label
    
def get_pose_from_image_and_bounding_box(bbox,frame):
    bbox = np.int32(bbox.clip(min=0))
    
    if len(bbox)>0 and np.all(bbox>0):#we do not want to cover the cases where we are the boundary of the image --> pose classification behaves strangely
        
        img = frame[bbox[1]:bbox[3], bbox[0]:bbox[2]]
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        

        #perform movenet on img(img is the cropped image)
        img1 = img.copy()
        img1 = tf.image.resize_with_pad(np.expand_dims(img1, axis=0), 192,192)
        input_image = tf.cast(img1, dtype=tf.float32)

        # Setup input and output 
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()

        # Make predictions 
        interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
        interpreter.invoke()
        keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])

        # Rendering 
        draw_connections(img, keypoints_with_scores, EDGES, 0.4)
        draw_keypoints(img, keypoints_with_scores, 0.4)
        
        print(keypoints_with_scores.reshape((17,3))[11,:])#prints x,y,confidence of left hip
        landmarks = keypoints_with_scores.reshape((17,3))[:,0:2]#array of landmarks (x,y)

        label = ""
        #Classification
        if np.size(landmarks) != 0:
            img, label = classifyPose(landmarks, img, display=False)
            
        
        cv2.imshow('Pose Landmarks',img)
    else:
        img = frame
        label = "no bounding box detected"
    return img, label

# Main Code

In [7]:
#Definition of the parameters
max_cosine_distance = 0.4 #The matching threshold. Samples with larger distance are considered an invalid match.
nn_budget = None #[int] If not None, fix samples per class to at most this number. Removes the oldest samples when the budget is reached.
nms_max_overlap = 1.0 # ROIs that overlap more than this values are suppressed.(non_max_suppression)

#Parameters for the Tracker():
max_age = 500#Maximum number of missed misses before a track is deleted.
n_init = 10
max_iou_distance = 0.7



# initialize deep sort
model_filename = 'model_data/mars-small128.pb'
encoder = gdet.create_box_encoder(model_filename, batch_size=1)
# calculate cosine distance metric
metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
# initialize tracker
tracker = Tracker(metric,max_iou_distance,max_age, n_init)

try:
    cap = cv2.VideoCapture(0)
except:
    print("webcam doesnt work")

initialized = False
frame_num = 0
first_pass = True

trigger_id = None

while True:
    return_value, frame = cap.read()
    if return_value:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(frame)
    else:
        print('Video has ended or failed, try a different video format!')
        break
    frame_num +=1
    
    #print('Frame #: ', frame_num)
    start_time = time.time()
    
    #yolo comes in place
    results = model(frame)#inference from yolo
    
    frame = results.render()[0]
    
    boxes, scores, names, imglist = get_bbox(results, frame)
    
    #frame, label = write_frame_keypoints(imglist,bboxes,frame)#movenet
    
    bboxes = format_boxes(boxes.transpose())# certain format of bboxes for the tracker
    
    features = encoder(frame, bboxes)
    detections = [Detection(bbox, score, class_name, feature) for bbox, score, class_name, feature in zip(bboxes, scores, names, features)]
    
    #initizialize color map
    cmap = plt.get_cmap('tab20b')    
    
    # run non-maxima supression
    boxs = np.array([d.tlwh for d in detections])
    scores = np.array([d.confidence for d in detections])
    classes = np.array([d.class_name for d in detections])
    indices = preprocessing.non_max_suppression(boxs, classes, nms_max_overlap, scores)
    detections = [detections[i] for i in indices]             
        
    #call the tracker
    tracker.predict()
    tracker.update(detections)
    #tracker.tracks = [t for t in tracker.tracks if (t.track_id ==1)] # this doesnt really work --> the features of other people are not taken into account
    
    
    colors = [cmap(i)[:3] for i in np.linspace(0, 1, 20)]
    # update tracks
    
    for track in tracker.tracks:
        if not track.is_confirmed() or track.time_since_update > 1:
            continue 
        bbox = track.to_tlbr()#sometimes the bounding boxes were negative
        class_name = track.get_class()
        #print(bbox)        
        
        img, label = get_pose_from_image_and_bounding_box(bbox,frame)
        if initialized == False:
        #pose estimation:
        
            #img, label = get_pose_from_image_and_bounding_box(bboxes.flatten(),frame)
        
            
        
            #print(label, "   first pass: " ,first_pass, "      initialized: ", initialized)
        
            #saving the POI ID
            if label == 'power to the people' and first_pass == True:
                trigger_id = track.track_id            
                tracker.tracks = [t for t in tracker.tracks if (t.track_id==trigger_id)]
                first_pass = False 
                initialized = True
                #cv2.destroyWindow('Pose Landmarks')
        
        if track.track_id == trigger_id:
            #if track.track_id == trigger_track.track_id:
            # draw bbox on screen
            #print("WE ARE IN THE SECOND LOOP")
            color = colors[int(track.track_id) % len(colors)]
            color = [i * 255 for i in color]
            cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), color, 2)
            cv2.rectangle(frame, (int(bbox[0]), int(bbox[1]-30)), (int(bbox[0])+(len(class_name)+len(str(track.track_id)))*17, int(bbox[1])), color, -1)
            cv2.putText(frame, class_name + "-" + str(track.track_id),(int(bbox[0]), int(bbox[1]-10)),0, 0.75, (255,255,255),2)
            #cv2.putText(frame, "My human",(int(bbox[0]), int(bbox[1]-10)),0, 0.75, (255,255,255),2)        
        
        
        
        
        #color = colors[int(track.track_id) % len(colors)]
        #color = [i * 255 for i in color]
        #cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), color, 2)
        #cv2.rectangle(frame, (int(bbox[0]), int(bbox[1]-30)), (int(bbox[0])+(len(class_name)+len(str(track.track_id)))*17, int(bbox[1])), color, -1)
        #cv2.putText(frame, class_name + "-" + str(track.track_id),(int(bbox[0]), int(bbox[1]-10)),0, 0.75, (255,255,255),2)        
        
    # calculate frames per second of running detections
    fps = 1.0 / (time.time() - start_time)
    #print("FPS: %.2f" % fps)
    result = np.asarray(frame)
    result = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR) 
    
    
    
    cv2.imshow('YoloV5 + Deep Sort', result)
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

[    0.99804     0.64234    0.012795]
[    0.99898     0.66016    0.014389]
[    0.99786     0.66288    0.017654]
[    0.99708     0.66381    0.013462]
[    0.91372     0.88486    0.016232]
[     0.3789     0.72187    0.019587]
[    0.37923     0.72415    0.018057]
[    0.37953     0.72492    0.019746]
[    0.36152     0.72391    0.019977]
[     1.0023      0.6592    0.012631]
[     1.0118     0.87215   0.0088421]
[     1.0172     0.69562   0.0054443]
[     1.0182     0.67548   0.0045443]
[     1.0187     0.69003   0.0051813]
[     1.0172     0.67276   0.0048825]
[     1.0182     0.67406   0.0050849]
[     1.0186     0.69089   0.0053991]
[     1.0173      0.6897   0.0052221]
[     1.0176     0.67472   0.0050552]
[     1.0166     0.70491   0.0050332]
[     1.0161     0.69492   0.0039679]
[     1.0144     0.69297   0.0038912]
[     1.0153       0.692   0.0039948]
[     1.0158     0.69192   0.0040958]
[     1.0152     0.69127    0.004966]
[    0.18329     0.69491     0.01973]
[     1.0152

In [8]:
cv2.destroyAllWindows()
cap.release()

In [9]:
img.shape

(480, 640, 3)

In [10]:
a = np.array([1,43,45,56])

In [11]:
a>0

array([ True,  True,  True,  True])

In [12]:
np.all(a>0)

True